In [1]:
import pandas as pd

In [62]:
data = pd.read_csv("parsl@parsl.csv")
data = data.to_numpy()

data_process = pd.read_csv("process_parsl.csv")
data_process = data_process.to_numpy()

In [63]:
# remove non-test build
print(len(data))
test_build_data = []

for row in data:
    if row[9] == True:
        test_build_data.append(row)

print(len(test_build_data))

5094
4401


In [90]:
# combine same build
unique_build = []

prev_build = test_build_data[0][0]
prev_commit = test_build_data[0][3]
test_string = str(test_build_data[0][18])

for i in range(1, len(test_build_data)+1):
    if i == len(test_build_data):
        unique_build.append([prev_build, prev_commit, test_string])
        break
    
    if test_build_data[i][0] == prev_build:
        test_string = test_string + "#" + str(test_build_data[i][18])
    else:
        unique_build.append([prev_build, prev_commit, test_string])
        prev_build = test_build_data[i][0]
        prev_commit = test_build_data[i][3]
        test_string = str(test_build_data[i][18])

# clear same tests
for i in range(len(unique_build)):
    temp_tests = []
    
    for item in unique_build[i][2].split("#"):
        if item !="nan":
            if item not in temp_tests:
                temp_tests.append(item)
    
    write_tests = ""
    for t in temp_tests:
        if write_tests == "":
            write_tests = write_tests + t
        else:
            write_tests = write_tests + "#" + t
        
    unique_build[i][2] = write_tests

print(unique_build)

[[382209089, '5de1201785d72a00b714f479f1799b6afddabd28', ''], [382233327, '0411f0030d53fb92043f40b236e0342aa9862649', ''], [382238855, 'd0da19cc61d5b4efe626b91a8e9c34613373ac78', ''], [382323312, 'aa18e11ac35ce94ad28dd8ce68a169c7ecc1c9e2', ''], [382323328, '3dcff4b75644c117ecd391551bb6f8e8806bda37', ''], [382326333, '08375ca6540329b069c5fe7895bde115a5f41229', ''], [382326344, '5eacb5f901c497b289d803f3f9f54b01cc5f7874', ''], [382387681, 'cc8b6dbbd28a69ecda2def1c3a7d5fcf0cb5cbff', ''], [382388152, '164bd0fb46175f69fc17d184587a2180d1a092a2', ''], [382442261, 'd0da19cc61d5b4efe626b91a8e9c34613373ac78', ''], [236466358, 'fb50cfae1434f8c16f84070dd016eff41f179f1b', ''], [382750127, 'ceb4fa96e8739549396a602e3e526dbd536f3773', ''], [382916806, '89a382d8fff3b596a0355d1e9112bdd0c3429098', ''], [382916953, '9023c1e0a2a030dbf9b4699e39b8eb6d5cab8a00', ''], [382918897, 'e1fbf897b8d2d3cd6de09c718a8cf78a4cd831c6', 'test_walltime[local]'], [382918914, '9e292045af6e7fdf207c6455c9ccc87ef848f6d8', ''], [38

In [98]:
# combine two data set to [build ID, commit, file_change, tests] - Commit Guru + Travis CI
combined_data = []

count_overlap = 0
count_no_overlap = 0
for i in range(len(unique_build)):
    findSame = False
    
    for j in range(len(data_process)):
        if unique_build[i][1] == data_process[j][0]:
            findSame = True
            new_row = [unique_build[i][0], unique_build[i][1], data_process[j][17], unique_build[i][2]]
            combined_data.append(new_row)
            break
    
    if findSame:
        count_overlap += 1
    else:
        count_no_overlap += 1

print("overlap: " + str(count_overlap) + " " + str(count_overlap/len(unique_build)))
print("non-overlap: " + str(count_no_overlap) + " " + str(count_no_overlap/len(unique_build)))

overlap: 1233 0.5541573033707865
non-overlap: 992 0.4458426966292135


In [122]:
# scrap file data from Github
import csv
import requests
from bs4 import BeautifulSoup as bs
import random
import time

with open("parsl_combined.csv", "w", newline='', encoding="utf-8") as f:
    csv_writer = csv.writer(f, delimiter=',')
    csv_writer.writerow(['build ID', 'commit', 'time', 'file change', 'tests'])
    
    for row in unique_build:
        build_id = row[0]
        commit_id = row[1]
        test_info = row[2]
        
        url = "https://github.com/Parsl/parsl/commit/" + commit_id
        print(url)
        
        r = requests.get(url)
        soup = bs(r.content, "html.parser")
        
        myDiv = soup.findAll('div', {'class': 'file-info flex-auto min-width-0 mb-md-0 mb-2'})
        
        temp_file_list = []
        for div in myDiv:
            temp_file_list.append(div.find('a').text)
            
        timeDiv = soup.find('relative-time')
        if timeDiv is not None:
            commit_time = timeDiv.get('datetime')
            print(commit_time)
        else:
            commit_time = ""
        
        output_file_string = ""
        for f in temp_file_list:
            if output_file_string == "":
                output_file_string = output_file_string + f
            else:
                output_file_string = output_file_string + "#" + f
        print(output_file_string)
        
        csv_writer.writerow([build_id, commit_id, commit_time, output_file_string, test_info])
        
        rand_int = random.randint(1, 4)
        time.sleep(rand_int)

https://github.com/Parsl/parsl/commit/5de1201785d72a00b714f479f1799b6afddabd28
2018-05-22T00:41:56Z
parsl/dataflow/usage_tracking/usage.py
https://github.com/Parsl/parsl/commit/0411f0030d53fb92043f40b236e0342aa9862649
2018-05-22T15:57:55Z
parsl/__init__.py
https://github.com/Parsl/parsl/commit/d0da19cc61d5b4efe626b91a8e9c34613373ac78
2018-05-22T16:07:48Z
parsl/tests/configs/cooley_ssh-il_single_node.py#parsl/tests/configs/osg_ipp_multinode.py#parsl/tests/configs/swan_ipp.py#parsl/tests/configs/theta_local_ipp_multinode.py
https://github.com/Parsl/parsl/commit/aa18e11ac35ce94ad28dd8ce68a169c7ecc1c9e2
2018-05-22T19:18:24Z
parsl/tests/test_regression/test_221.py
https://github.com/Parsl/parsl/commit/3dcff4b75644c117ecd391551bb6f8e8806bda37
2018-05-22T19:19:38Z
parsl/dataflow/dflow.py#parsl/tests/conftest.py#parsl/tests/test_bash_apps/test_basic.py#parsl/tests/test_regression/test_221.py
https://github.com/Parsl/parsl/commit/08375ca6540329b069c5fe7895bde115a5f41229
2018-05-22T19:26:07Z
par

2018-05-31T21:21:26Z
parsl/data_provider/data_manager.py#parsl/data_provider/files.py#parsl/data_provider/globus.py#parsl/dataflow/config_defaults.py
https://github.com/Parsl/parsl/commit/14b7c0bd734d4a5be0d4c98b57edef601299de75
2018-05-31T21:27:44Z
parsl/data_provider/data_manager.py#parsl/dataflow/dflow.py
https://github.com/Parsl/parsl/commit/9e802ed1f82e037068fe468a206c96a2b081a0c4
2018-05-31T21:27:54Z
parsl/data_provider/data_manager.py#parsl/data_provider/files.py#parsl/data_provider/globus.py#parsl/dataflow/config_defaults.py
https://github.com/Parsl/parsl/commit/547ef33559003eddb8206bb01d9bbc22bc07aba7
2018-06-01T11:24:38Z
parsl/data_provider/data_manager.py#parsl/dataflow/config_defaults.py
https://github.com/Parsl/parsl/commit/a8209666b78231bef2b215af05b1657137869484
2018-06-01T11:24:45Z
parsl/data_provider/data_manager.py#parsl/data_provider/files.py#parsl/data_provider/globus.py#parsl/dataflow/config_defaults.py
https://github.com/Parsl/parsl/commit/ab2feb4bf323265539cdb2e2

2017-02-25T07:37:38Z
README.rst#docs/_build/doctrees/dev_docs.doctree#docs/_build/doctrees/devguide/design.doctree#docs/_build/doctrees/devguide/dev_docs.doctree#docs/_build/doctrees/environment.pickle#docs/_build/doctrees/index.doctree#docs/_build/doctrees/quick/quickstart.doctree#docs/_build/doctrees/quick/tutorial.doctree#docs/_build/doctrees/tut/tutorial.doctree#docs/_build/doctrees/userguide/apps.doctree#docs/_build/doctrees/userguide/futures.doctree#docs/_build/doctrees/userguide/overview.doctree#docs/_build/html/.buildinfo#docs/_build/html/.nojekyll#docs/_build/html/_sources/dev_docs.txt#docs/_build/html/_sources/devguide/design.txt#docs/_build/html/_sources/devguide/dev_docs.txt#docs/_build/html/_sources/index.txt#docs/_build/html/_sources/quick/quickstart.txt#docs/_build/html/_sources/quick/tutorial.txt#docs/_build/html/_sources/tut/tutorial.txt#docs/_build/html/_sources/userguide/apps.txt
https://github.com/Parsl/parsl/commit/99efafe2bb299c918c52b856b6f20209ffb4d34d
2018-06-0

https://github.com/Parsl/parsl/commit/d896d74468582cff701abfe07b9ee4eca78b850c
2018-06-20T15:05:04Z
test-requirements.txt
https://github.com/Parsl/parsl/commit/6199c1ad4c2c6af724663c855e1249272d856367
2018-06-20T15:05:35Z
requirements.txt#test-requirements.txt
https://github.com/Parsl/parsl/commit/78453edafcc824bc95c95cb6d218f7eb753c021c
2018-06-20T15:12:46Z
requirements.txt#test-requirements.txt
https://github.com/Parsl/parsl/commit/8e761b45443ab34eacb67d6b38b6976c338fb256
2018-06-21T02:42:03Z
parsl/app/app.py
https://github.com/Parsl/parsl/commit/8e5cf407ca11951d1d23d245d7c33ea2158881e3
2018-06-21T02:45:17Z
parsl/app/app.py
https://github.com/Parsl/parsl/commit/6da3dfc2789e53a0a97665dbee52092b8a25c36f
2018-06-21T03:05:49Z
parsl/app/app.py
https://github.com/Parsl/parsl/commit/ce3ce441645b0f92ef36ffeb925001efe4bf0558

https://github.com/Parsl/parsl/commit/ccded0208da66da212af54c51999625453f221de
2018-06-21T03:06:13Z
parsl/app/app.py
https://github.com/Parsl/parsl/commit/06afb8f25623c2

2018-06-29T22:19:28Z
requirements.txt
https://github.com/Parsl/parsl/commit/b50b73914a680671511c3c4fb1e91842d3dd92c3
2018-06-29T22:19:36Z
docs/conf.py#docs/devguide/dev_docs.rst#docs/faq.rst#docs/reference.rst#docs/userguide/app_caching.rst
https://github.com/Parsl/parsl/commit/000bd1d38fce2bf63444c0586e4067796c525039
2018-06-30T04:37:13Z
parsl/tests/test_docker_multisite.py
https://github.com/Parsl/parsl/commit/23040bc19a66621b3c77aaae4268ac64102fde20
2018-06-30T04:37:35Z
docs/conf.py#docs/devguide/dev_docs.rst#docs/faq.rst#docs/reference.rst#docs/userguide/app_caching.rst
https://github.com/Parsl/parsl/commit/3b622f96c16d62a69c04f465808f0b3b80ed0cd3
2018-06-30T04:54:41Z
parsl/tests/configs/local_ipp_multisite.py#parsl/tests/configs/local_threads_ipp.py
https://github.com/Parsl/parsl/commit/3b35eb04d30f03bac873446d69fd08bd1623d42d
2018-06-30T04:55:11Z
docs/conf.py#docs/devguide/dev_docs.rst#docs/faq.rst#docs/reference.rst#docs/userguide/app_caching.rst
https://github.com/Parsl/parsl/c

...

2018-04-26T16:43:10Z
parsl/dataflow/dflow.py
https://github.com/Parsl/parsl/commit/9ae4a9eb3585c1d17d784ad250e57e2e3eea64d7
2018-04-26T16:43:40Z
parsl/dataflow/dflow.py#parsl/tests/test_checkpointing/test_regression_233.py#parsl/tests/test_checkpointing/test_task_exit.py#parsl/tests/test_cleanup/test_cleanup.py#parsl/tests/test_sites/test_ec2/ec2.py#parsl/tests/test_sites/test_ec2/ec2.py
https://github.com/Parsl/parsl/commit/d84e8bbc7879ea033f3b04733f50cc5495c48d5c
2018-04-26T20:31:43Z
parsl/app/app.py#parsl/app/bash_app.py
https://github.com/Parsl/parsl/commit/63bf77ccade215d58e7bf8ccdd0b9e9d4ad9c310
2018-04-26T20:38:37Z
.travis.yml#parsl/app/app.py#parsl/app/bash_app.py#parsl/tests/test_regression/test_226.py#test-requirements.txt
https://github.com/Parsl/parsl/commit/ee5388cf503e4f5a3d046143bf3559f620792429
2018-04-26T20:41:41Z
parsl/app/app.py#parsl/app/bash_app.py
https://github.com/Parsl/parsl/commit/28c9bc21ef374e6bb7d2b41bcca55bb6a62c58f5
2018-04-26T20:42:33Z
.travis.yml#parsl/

2018-05-03T17:59:18Z
README.rst
https://github.com/Parsl/parsl/commit/a8dcfa4efc52c2dcd57310badb18bd740022979a
2018-05-03T18:03:07Z
docs/userguide/containers.rst
https://github.com/Parsl/parsl/commit/8ee1f4be8aa72842cfe483354d8818c8d5f17378
2018-05-03T18:07:49Z
README.rst
https://github.com/Parsl/parsl/commit/fad0f3edd11f8273aa3d2b18e2913bbf4e3b2313
2017-04-04T02:17:37Z
sti/pyc/README.adoc
https://github.com/Parsl/parsl/commit/e64b9747480c99859a31260903bd6b3775b5041a
2018-05-03T18:07:58Z
README.rst
https://github.com/Parsl/parsl/commit/2e3ea44ef07612167c4bf3dc6abc69e5eea9281e
2018-05-03T18:14:15Z
README.rst
https://github.com/Parsl/parsl/commit/bffa6e5a6062dc4f89da628bca7d2199444143d6
2018-05-03T19:46:06Z
parsl/dataflow/dflow.py
https://github.com/Parsl/parsl/commit/abada619227376cadead36db68a0df1568ddbb42
2018-05-03T19:46:46Z
parsl/dataflow/dflow.py#parsl/tests/test_checkpointing/test_dfk_exit.py#parsl/tests/test_checkpointing/test_regression_233.py#parsl/tests/test_checkpointing/test

2018-05-10T09:38:41Z
README.rst#docs/faq.rst#docs/quickstart.rst#docs/userguide/app_caching.rst#docs/userguide/checkpoints.rst#docs/userguide/data.rst#docs/userguide/exceptions.rst#docs/userguide/execution.rst#docs/userguide/overview.rst#parsl/app/bash_app.py#parsl/dataflow/dflow.py#parsl/dataflow/memoization.py#parsl/executors/base.py
https://github.com/Parsl/parsl/commit/02d99288b32134544d0eebea65f65f9103e34165
2018-05-10T09:47:56Z
parsl/dataflow/dflow.py
https://github.com/Parsl/parsl/commit/d116c171e1360c04c7ed29267f44d83c602712c5
2018-05-10T09:48:40Z
README.rst#docs/faq.rst#docs/quickstart.rst#docs/userguide/app_caching.rst#docs/userguide/checkpoints.rst#docs/userguide/data.rst#docs/userguide/exceptions.rst#docs/userguide/execution.rst#docs/userguide/overview.rst#parsl/app/bash_app.py#parsl/dataflow/dflow.py#parsl/dataflow/memoization.py#parsl/executors/base.py
https://github.com/Parsl/parsl/commit/52b23d6dd261bec11be21351460f56d2a864f01f
2018-05-10T14:59:55Z
README.rst#docs/faq.r

https://github.com/Parsl/parsl/commit/0126e7b3e979d3714db6d9551e9dc8c998c13404
2018-05-11T20:10:12Z
parsl/dataflow/dflow.py#parsl/dataflow/usage_tracking/usage.py#parsl/tests/test_checkpointing/checkpointed.py#parsl/tests/test_checkpointing/test_regression_232.py#parsl/tests/test_manual/test_regression_220.py
https://github.com/Parsl/parsl/commit/e168e86f4dba6a05582e91e81ec51bbdab27493b
2018-05-11T20:16:06Z
parsl/dataflow/dflow.py#parsl/dataflow/usage_tracking/usage.py#parsl/tests/test_checkpointing/checkpointed.py#parsl/tests/test_checkpointing/test_regression_232.py#parsl/tests/test_manual/test_regression_220.py
https://github.com/Parsl/parsl/commit/fbcb960c6c49f4d92779923f302e87c753278119
2018-05-11T20:20:49Z
parsl/integration_tests/latency.py → parsl/tests/integration/latency.py#...t_apps/HEDM_processlayer/process_layer.py → ...t_apps/HEDM_processlayer/process_layer.py#...l/integration_tests/test_apps/__init__.py → ...l/tests/integration/test_apps/__init__.py#...tion_tests/test_app

https://github.com/Parsl/parsl/commit/012d8881f43f8e1dfd0f40b850e4261589241f0b
2018-05-15T16:38:07Z
parsl/tests/test_checkpointing/test_regression_232.py
https://github.com/Parsl/parsl/commit/055ae65f76baa70fd798c601fd3aaa799d6a1e5d
2018-05-15T16:38:36Z
.flake8#.travis.yml#CONTRIBUTING.rst#docs/userguide/execution.rst#parsl/__init__.py#parsl/configs/local.py#parsl/configs/local_multi.py#parsl/dataflow/dflow.py#parsl/dataflow/rundirs.py#parsl/tests/cleanup
https://github.com/Parsl/parsl/commit/3871d944704629d5745855c49a801f4b66255525
2018-05-15T18:03:45Z
.travis.yml#parsl/tests/test_checkpointing/test_regression_232.py
https://github.com/Parsl/parsl/commit/928dbe7f7d085198c67894ac88486246fd17562e
2018-05-15T18:04:16Z
.flake8#.travis.yml#CONTRIBUTING.rst#docs/userguide/execution.rst#parsl/__init__.py#parsl/configs/local.py#parsl/configs/local_multi.py#parsl/dataflow/dflow.py#parsl/dataflow/rundirs.py#parsl/tests/cleanup
https://github.com/Parsl/parsl/commit/9fbe12da8b395ab681741328ad3c7d

In [137]:
combined_data = pd.read_csv("parsl_combined.csv")
combined_data = combined_data.to_numpy()

failed_data = []
for row in combined_data:
    if str(row[4]) != "nan":
        failed_data.append(row)

print(len(combined_data))
print(len(failed_data))

2195
193


In [138]:
# train test split
tot_len = len(failed_data)
testing_len = int(tot_len * 0.7)
testing_data = failed_data[testing_len:]
print(testing_len)

split_build_id = failed_data[testing_len][0]

training_data = []

for row in combined_data:
    if row[0] == split_build_id:
        break
    
    training_data.append(row)

print(len(training_data))
print(len(testing_data))

135
1500
58


In [140]:
# create File-Commit-Timestep Table
file_list = []
for row in training_data:
    temp_file = row[3]
    
    for f in temp_file.split("#"):
        if f not in file_list:
            file_list.append(f)

file_commit_table = []

for row in training_data:
    cur_list = []
    cur_list.append(row[1])
    
    for file in file_list:
        if file in row[3].split('#'):
            cur_list.append(1)
        else:
            cur_list.append(0)
    
    cur_list.append(row[2])
    
    file_commit_table.append(cur_list)

print(file_commit_table)

[['e644453b63e37dce6410f6c3530cacbfb70ad431', 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [142]:
# collect test cases, build test-file Table
test_list = []
for row in failed_data:
    temp_test = row[4]
    
    for t in temp_test.split("#"):
        if t not in test_list:
            test_list.append(t)

test_file_table = [[0 for i in range(len(test_list))] for j in range(len(file_list))]
for row in training_data:
    if str(row[4]) != "nan":
        for i in range(len(file_list)):
            if file_list[i] in row[3].split("#"):
                for j in range(len(test_list)):
                    if test_list[j] in row[4].split("#"):
                        test_file_table[i][j] += 1

print(test_file_table)           

[[1, 1, 1, 1, 1, 1, 0, 0, 0, 4, 4, 4, 4, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 2, 1, 1, 0, 2, 2, 3, 2, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [174]:
# timestamp for each commit
import datetime

list_timestamp = []
list_commit = []
for row in file_commit_table:
    list_timestamp.append(row[-1])
    list_commit.append(row[1])

original_date = datetime.datetime(int(list_timestamp[0][0:4]), int(list_timestamp[0][5:7]), int(list_timestamp[0][8:10]), 
                                  int(list_timestamp[0][11:13]), int(list_timestamp[0][14:16]), int(list_timestamp[0][17:19]))

norm_timestamp = []
for item in list_timestamp:
    temp_date = datetime.datetime(int(item[0:4]), int(item[5:7]), int(item[8:10]),
                                  int(item[11:13]), int(item[14:16]), int(item[17:19]))
    
    diff = temp_date - original_date
    diff_sec = diff.total_seconds()
    score = diff_sec * 1
    norm_timestamp.append(score+5)

file_score = {}
for f in file_list:
    file_score.update({f: 0})

for i in range(len(training_data)):
    tc = norm_timestamp[i]/max(norm_timestamp)
    
    for f in training_data[i][3].split("#"):
        file_score[f] += tc

print(file_score)   

{'.travis.yml': 42.78262975761239, 'docs/index.rst': 21.635192388195353, 'setup.py': 21.790266428325562, 'parsl/tests/test_factory.py': 5.639375009574391, 'parsl/tests/test_python_apps.py': 6.24875532922304e-05, 'test-requirements.txt': 6.899561946228971, 'docs/conf.py': 40.98847890132577, 'README.rst': 37.51180143292282, 'README.md': 0.00045322038730871933, 'parsl/app/app.py': 53.360572350816916, 'docs/_build/doctrees/dev_docs.doctree': 0.0008843024388200349, 'docs/_build/doctrees/devguide/design.doctree': 0.0008843024388200349, 'docs/_build/doctrees/devguide/dev_docs.doctree': 0.0008843024388200349, 'docs/_build/doctrees/environment.pickle': 0.0008843024388200349, 'docs/_build/doctrees/index.doctree': 0.0008843024388200349, 'docs/_build/doctrees/quick/quickstart.doctree': 0.0008843024388200349, 'docs/_build/doctrees/quick/tutorial.doctree': 0.0008843024388200349, 'docs/_build/doctrees/tut/tutorial.doctree': 0.0008843024388200349, 'docs/_build/doctrees/userguide/apps.doctree': 0.00088

In [ ]:
# test on test data set
